In [2]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

# Set up Splinter
browser = Browser('chrome')

# Create an empty list to store the movie data
movies = []

# Loop over all the URLs
for i in range(10):
    # Construct the URL for the current page
    url = f'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start={i*100+1}&ref_=adv_nxt'

    # Visit the URL
    browser.visit(url)

    # Parse the website
    html = browser.html
    html_soup = soup(html, 'html.parser')

    # Find the list of movies
    movie_list = html_soup.find('div', {'class': 'lister-list'})

    # Find all the rows in the list
    movie_rows = movie_list.find_all('div', {'class': 'lister-item'})

    # Loop over the rows and extract the movie details
    for row in movie_rows:
        # Get the movie title
        title = row.find('h3', {'class': 'lister-item-header'}).find('a').text

        # Get the movie year
        year = row.find('span', {'class': 'lister-item-year'}).text.strip('()')

        # Get the movie rating
        rating = row.find('div', {'class': 'ratings-bar'}).find('strong').text

        # Get the movie director, stars, votes, and gross
        details = row.find('div', {'class': 'lister-item-content'})
        director = details.find_all('p')[2].find_all('a')[0].text
        stars = [star.text for star in details.find_all('p')[2].find_all('a')[1:]]
        votes = details.find('p', {'class': 'sort-num_votes-visible'}).find_all('span')[1].text.replace(',', '')
        gross = details.find('p', {'class': 'sort-num_votes-visible'}).find_all('span')[-1].text.replace('$', '').replace(',', '').strip()
        genres = details.find('span', {'class': 'genre'}).text.strip().split(', ')
        
        # Check if gross value is in millions or thousands and convert accordingly
        if gross.endswith('M'):
            gross = float(gross.strip('M')) * 1000000
        elif gross.endswith('K'):
            gross = float(gross.strip('K')) * 1000
            
        # Add the movie data to the list
        movies.append({'Title': title, 'Year': year, 'Rating': rating, 'Director': director, 'Stars': stars, 'Votes': votes, 'Gross': gross, 'Genres': genres})

# Create a DataFrame from the movie data
df = pd.DataFrame(movies)

print(df)

# Export the DataFrame to a CSV file
df.to_csv('imdb_top_1000_movies_final.csv', index=False)


                                             Title  Year Rating  \
0                         The Shawshank Redemption  1994    9.3   
1                                    The Godfather  1972    9.2   
2                                  The Dark Knight  2008    9.0   
3    The Lord of the Rings: The Return of the King  2003    9.0   
4                                 Schindler's List  1993    9.0   
..                                             ...   ...    ...   
995                                          Shine  1996    7.6   
996                            Eyes Without a Face  1960    7.6   
997                                 The Odd Couple  1968    7.6   
998                              The Invisible Man  1933    7.6   
999                                      Celda 211  2009    7.6   

                 Director                                              Stars  \
0          Frank Darabont  [Tim Robbins, Morgan Freeman, Bob Gunton, Will...   
1    Francis Ford Coppola  [Marlon 